In [1]:
#AthMethPre Xiang, S., et al., AthMethPre: a web server for the prediction and query of mRNA m 6 A sites in Arabidopsis thaliana. Molecular BioSystems, 2016. 12(11): p. 3333-3337.
import pandas as pd
import numpy as np
import itertools
import os
import sys

dataset_name="trainset"
gene_type="DNA"
type_value="U"
if gene_type=="RNA":
    type_value="U"
elif gene_type=="DNA":
    type_value="T"

def read_fasta_file():
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh = open('seq.txt', 'r')
    seq = []
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\n', '').replace('\r', ''))
    fh.close()
    matrix_data = np.array([list(e) for e in seq])
    print(matrix_data)
    return matrix_data

def AthMethPre_extract_one_line(data_line):
    '''
    extract features from one line, such as one m6A sample
    '''
    A=[0,0,0,1]
    T=[0,0,1,0]
    C=[0,1,0,0]
    G=[1,0,0,0]
    N=[0,0,0,0]
    feature_representation={"A":A,"C":C,"G":G,"N":N}
    feature_representation[type_value]=T
    beginning=0
    end=len(data_line)-1
    one_line_feature=[]
    alphabet='ACNG'
    alphabet+=type_value
    matrix_two=["".join(e) for e in itertools.product(alphabet, repeat=2)] # AA AU AC AG UU UC ...
    matrix_three=["".join(e) for e in itertools.product(alphabet, repeat=3)]# AAA AAU AAC ...
    matrix_four=["".join(e) for e in itertools.product(alphabet, repeat=4)]# AAAA AAAU AAAC ...
    feature_two=np.zeros(25)
    feature_three=np.zeros(125)
    feature_four=np.zeros(625)
    for index,data in enumerate(data_line):
        if index==beginning or index==end:
            one_line_feature.extend(feature_representation["N"])
        elif data in feature_representation.keys():
            one_line_feature.extend(feature_representation["N"])
            one_line_feature.extend(feature_representation[data])
        if "".join(data_line[index:(index+2)]) in matrix_two and index <= end-1:
            feature_two[matrix_two.index("".join(data_line[index:(index+2)]))]+=1
        if "".join(data_line[index:(index+3)]) in matrix_three and index <= end-2:
            feature_three[matrix_three.index("".join(data_line[index:(index+3)]))]+=1
        if "".join(data_line[index:(index+4)]) in matrix_four and index <=end-3:
            feature_four[matrix_four.index("".join(data_line[index:(index+4)]))]+=1
    sum_two=np.sum(feature_two)
    sum_three=np.sum(feature_three)
    sum_four=np.sum(feature_four)
    one_line_feature.extend(feature_two/sum_two)
    one_line_feature.extend(feature_three/sum_three)
    one_line_feature.extend(feature_four/sum_four)
    return one_line_feature


def AthMethPre_extract_one_line_without(data_line):
    '''
    extract features from one line, such as one m6A sample
    '''
    A=[1,0,0,0]
    C=[0,1,0,0]
    G=[0,0,1,0]
    T=[0,0,0,1]
    feature_representation={"A":A,"C":C,"G":G,"T":T}
    beginning=0
    end=len(data_line)-1
    one_line_feature=[]
    alphabet='ACGT'
    matrix_two=["".join(e) for e in itertools.product(alphabet, repeat=2)] # AA AU AC AG UU UC ...
    matrix_three=["".join(e) for e in itertools.product(alphabet, repeat=3)]# AAA AAU AAC ...
    matrix_four=["".join(e) for e in itertools.product(alphabet, repeat=4)]# AAAA AAAU AAAC ...
    feature_two=np.zeros(16)
    feature_three=np.zeros(64)
    feature_four=np.zeros(256)
    for index,data in enumerate(data_line):
        one_line_feature.extend(feature_representation[data])
        if "".join(data_line[index:(index+2)]) in matrix_two and index <= end-1:
            feature_two[matrix_two.index("".join(data_line[index:(index+2)]))]+=1
        if "".join(data_line[index:(index+3)]) in matrix_three and index <= end-2:
            feature_three[matrix_three.index("".join(data_line[index:(index+3)]))]+=1
        if "".join(data_line[index:(index+4)]) in matrix_four and index <=end-3:
            feature_four[matrix_four.index("".join(data_line[index:(index+4)]))]+=1
    sum_two=np.sum(feature_two)
    sum_three=np.sum(feature_three)
    sum_four=np.sum(feature_four)
    one_line_feature.extend(feature_two/sum_two)
    one_line_feature.extend(feature_three/sum_three)
    one_line_feature.extend(feature_four/sum_four)
    return one_line_feature


def AthMethPre_feature_extraction(matrix_data,fill_NA):
    if fill_NA=="1":
        final_feature_matrix=[AthMethPre_extract_one_line(e) for e in matrix_data]
    elif fill_NA=="0":
        final_feature_matrix=[AthMethPre_extract_one_line_without(e) for e in matrix_data]
    return final_feature_matrix


fill_NA="0"
matrix_data=read_fasta_file()
final_feature_matrix=AthMethPre_feature_extraction(matrix_data,fill_NA)
print(np.array(final_feature_matrix).shape)
#pd.DataFrame(final_feature_matrix).to_csv('Result.csv',header=None,index=False)


[['A' 'C' 'G' ... 'T' 'T' 'T']
 ['A' 'C' 'T' ... 'C' 'G' 'C']
 ['T' 'G' 'G' ... 'G' 'A' 'T']
 ...
 ['C' 'A' 'T' ... 'G' 'C' 'G']
 ['G' 'A' 'C' ... 'G' 'A' 'G']
 ['G' 'C' 'T' ... 'A' 'G' 'T']]
(3108, 500)


In [5]:
    data_line=matrix_data[0]
    A=[1,0,0,0]
    C=[0,1,0,0]
    G=[0,0,1,0]
    T=[0,0,0,1]
    feature_representation={"A":A,"C":C,"G":G,"T":T}
    beginning=0
    end=len(data_line)-1
    one_line_feature=[]
    alphabet='ACGT'
    matrix_two=["".join(e) for e in itertools.product(alphabet, repeat=2)] # AA AU AC AG UU UC ...
    matrix_three=["".join(e) for e in itertools.product(alphabet, repeat=3)]# AAA AAU AAC ...
    matrix_four=["".join(e) for e in itertools.product(alphabet, repeat=4)]# AAAA AAAU AAAC ...
    feature_two=np.zeros(16)
    feature_three=np.zeros(64)
    feature_four=np.zeros(256)
    for index,data in enumerate(data_line):
        one_line_feature.extend(feature_representation[data])
        if "".join(data_line[index:(index+2)]) in matrix_two and index <= end-1:
            feature_two[matrix_two.index("".join(data_line[index:(index+2)]))]+=1
        if "".join(data_line[index:(index+3)]) in matrix_three and index <= end-2:
            feature_three[matrix_three.index("".join(data_line[index:(index+3)]))]+=1
        if "".join(data_line[index:(index+4)]) in matrix_four and index <=end-3:
            feature_four[matrix_four.index("".join(data_line[index:(index+4)]))]+=1
    sum_two=np.sum(feature_two)
    sum_three=np.sum(feature_three)
    sum_four=np.sum(feature_four)
    one_line_feature.extend(feature_two/sum_two)
    one_line_feature.extend(feature_three/sum_three)
    one_line_feature.extend(feature_four/sum_four)
    print('data_line=',data_line)
    print('matrix_two=',matrix_two)
    print('feature_two=',feature_two)
    print('sum_two=',sum_two)
    print(one_line_feature)
    


data_line= ['A' 'C' 'G' 'T' 'G' 'A' 'G' 'C' 'T' 'G' 'A' 'T' 'G' 'A' 'C' 'T' 'C' 'A'
 'C' 'A' 'C' 'T' 'T' 'A' 'C' 'T' 'A' 'G' 'G' 'C' 'A' 'T' 'T' 'C' 'C' 'T'
 'C' 'G' 'T' 'T' 'T']
matrix_two= ['AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT']
feature_two= [0. 5. 2. 2. 3. 1. 2. 5. 3. 2. 1. 2. 2. 3. 3. 4.]
sum_two= 40.0
[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0.0, 0.125, 0.05, 0.05, 0.075, 0.025, 0.05, 0.125, 0.075, 0.05, 0.025, 0.05, 0.05, 0.075, 0.075, 0.1, 0.0, 0.0, 0.0, 0.0, 0.02564102564102564,

In [10]:
matrix_data

array([['A', 'C', 'G', ..., 'T', 'T', 'T'],
       ['A', 'C', 'T', ..., 'C', 'G', 'C'],
       ['T', 'G', 'G', ..., 'G', 'A', 'T'],
       ...,
       ['C', 'A', 'T', ..., 'G', 'C', 'G'],
       ['G', 'A', 'C', ..., 'G', 'A', 'G'],
       ['G', 'C', 'T', ..., 'A', 'G', 'T']], dtype='<U1')

In [12]:
for index_1,data_1 in enumerate(matrix_data):
    print(index_1,data_1)

0 ['A' 'C' 'G' 'T' 'G' 'A' 'G' 'C' 'T' 'G' 'A' 'T' 'G' 'A' 'C' 'T' 'C' 'A'
 'C' 'A' 'C' 'T' 'T' 'A' 'C' 'T' 'A' 'G' 'G' 'C' 'A' 'T' 'T' 'C' 'C' 'T'
 'C' 'G' 'T' 'T' 'T']
1 ['A' 'C' 'T' 'T' 'T' 'C' 'A' 'C' 'A' 'C' 'C' 'G' 'A' 'C' 'G' 'T' 'C' 'A'
 'C' 'A' 'C' 'A' 'C' 'A' 'C' 'T' 'A' 'G' 'C' 'G' 'T' 'A' 'A' 'C' 'T' 'A'
 'A' 'A' 'C' 'G' 'C']
2 ['T' 'G' 'G' 'T' 'A' 'A' 'C' 'T' 'T' 'T' 'T' 'C' 'T' 'G' 'A' 'C' 'A' 'C'
 'C' 'T' 'C' 'T' 'T' 'A' 'C' 'C' 'T' 'G' 'A' 'A' 'A' 'C' 'A' 'A' 'T' 'C'
 'A' 'G' 'G' 'A' 'T']
3 ['T' 'T' 'T' 'G' 'A' 'C' 'T' 'C' 'A' 'A' 'C' 'A' 'C' 'G' 'G' 'G' 'A' 'A'
 'A' 'A' 'C' 'T' 'C' 'A' 'C' 'C' 'C' 'G' 'G' 'T' 'C' 'C' 'G' 'G' 'A' 'C'
 'A' 'C' 'C' 'A' 'T']
4 ['T' 'G' 'A' 'T' 'A' 'C' 'T' 'T' 'T' 'C' 'G' 'G' 'G' 'T' 'T' 'G' 'A' 'T'
 'G' 'G' 'C' 'G' 'G' 'A' 'C' 'T' 'A' 'G' 'T' 'G' 'A' 'T' 'T' 'G' 'T' 'G'
 'C' 'T' 'T' 'G' 'C']
5 ['T' 'G' 'C' 'C' 'G' 'A' 'G' 'A' 'C' 'G' 'C' 'G' 'A' 'A' 'T' 'C' 'T' 'C'
 'A' 'G' 'C' 'T' 'C' 'A' 'C' 'T' 'A' 'A' 'T' 'G' 'T' 'T' 'G' 'C' 'C' 'C'
 '

 'G' 'C' 'A' 'G' 'A']
776 ['C' 'A' 'G' 'T' 'A' 'A' 'A' 'G' 'C' 'G' 'G' 'G' 'G' 'G' 'T' 'C' 'G' 'A'
 'A' 'C' 'C' 'T' 'A' 'C' 'A' 'T' 'T' 'G' 'T' 'G' 'T' 'G' 'A' 'G' 'C' 'G'
 'A' 'C' 'A' 'A' 'T']
777 ['C' 'A' 'G' 'T' 'T' 'C' 'G' 'A' 'C' 'T' 'G' 'T' 'T' 'A' 'G' 'A' 'G' 'A'
 'C' 'G' 'C' 'T' 'A' 'A' 'A' 'T' 'A' 'A' 'G' 'A' 'A' 'A' 'A' 'T' 'T' 'G'
 'G' 'C' 'G' 'G' 'A']
778 ['C' 'T' 'T' 'C' 'C' 'G' 'G' 'A' 'A' 'C' 'A' 'T' 'T' 'A' 'A' 'A' 'A' 'A'
 'C' 'G' 'C' 'A' 'T' 'A' 'A' 'A' 'C' 'G' 'T' 'T' 'A' 'G' 'C' 'G' 'C' 'C'
 'C' 'A' 'A' 'G' 'T']
779 ['G' 'T' 'G' 'G' 'C' 'A' 'C' 'A' 'A' 'G' 'A' 'A' 'A' 'G' 'C' 'C' 'C' 'C'
 'C' 'C' 'C' 'T' 'T' 'A' 'C' 'C' 'T' 'C' 'A' 'T' 'A' 'A' 'G' 'A' 'C' 'T'
 'T' 'C' 'G' 'A' 'A']
780 ['A' 'T' 'T' 'C' 'A' 'A' 'T' 'G' 'C' 'C' 'C' 'A' 'T' 'A' 'C' 'A' 'T' 'G'
 'A' 'G' 'C' 'A' 'T' 'A' 'A' 'T' 'A' 'G' 'T' 'C' 'C' 'A' 'G' 'C' 'A' 'A'
 'T' 'T' 'G' 'A' 'T']
781 ['C' 'C' 'A' 'A' 'A' 'T' 'T' 'C' 'T' 'G' 'A' 'G' 'A' 'T' 'T' 'T' 'T' 'T'
 'C' 'C' 'C' 'A' 'T' 'A' 'A' 'A' 'T' 'G' 

 'C' 'A' 'T' 'G' 'C']
1056 ['A' 'G' 'T' 'A' 'A' 'A' 'A' 'A' 'C' 'A' 'G' 'A' 'G' 'G' 'T' 'T' 'T' 'C'
 'C' 'G' 'C' 'G' 'G' 'A' 'A' 'A' 'T' 'T' 'T' 'T' 'A' 'T' 'G' 'G' 'G' 'A'
 'G' 'C' 'A' 'T' 'G']
1057 ['A' 'T' 'C' 'A' 'G' 'T' 'T' 'G' 'A' 'T' 'A' 'T' 'T' 'T' 'T' 'T' 'T' 'C'
 'C' 'A' 'C' 'C' 'T' 'G' 'A' 'T' 'A' 'A' 'C' 'G' 'T' 'T' 'T' 'A' 'C' 'A'
 'G' 'G' 'A' 'A' 'C']
1058 ['G' 'A' 'T' 'G' 'G' 'A' 'A' 'T' 'C' 'A' 'T' 'C' 'G' 'G' 'G' 'A' 'G' 'T'
 'A' 'C' 'C' 'T' 'T' 'G' 'G' 'C' 'C' 'G' 'T' 'A' 'G' 'C' 'A' 'A' 'T' 'C'
 'T' 'G' 'T' 'G' 'T']
1059 ['T' 'G' 'T' 'C' 'C' 'G' 'T' 'A' 'T' 'T' 'T' 'G' 'C' 'G' 'A' 'C' 'A' 'A'
 'G' 'T' 'C' 'T' 'T' 'A' 'C' 'T' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'A' 'T' 'G'
 'G' 'A' 'A' 'G' 'G']
1060 ['T' 'A' 'T' 'T' 'A' 'T' 'A' 'G' 'A' 'A' 'A' 'A' 'G' 'C' 'A' 'T' 'T' 'G'
 'G' 'A' 'C' 'C' 'C' 'C' 'G' 'A' 'T' 'T' 'C' 'T' 'G' 'G' 'A' 'T' 'A' 'G'
 'T' 'A' 'A' 'G' 'T']
1061 ['A' 'G' 'C' 'T' 'G' 'T' 'T' 'T' 'T' 'A' 'T' 'A' 'G' 'A' 'C' 'G' 'A' 'T'
 'C' 'G' 'C' 'G' 'G' 'G' 'G' 'T' 'G

 'C' 'C' 'C' 'A' 'T']
1568 ['T' 'T' 'T' 'A' 'C' 'G' 'A' 'G' 'C' 'T' 'G' 'A' 'A' 'T' 'A' 'T' 'T' 'T'
 'T' 'C' 'C' 'T' 'T' 'C' 'T' 'A' 'T' 'T' 'T' 'T' 'T' 'A' 'T' 'T' 'T' 'G'
 'A' 'T' 'T' 'T' 'G']
1569 ['A' 'A' 'A' 'C' 'A' 'A' 'T' 'T' 'C' 'C' 'A' 'T' 'T' 'A' 'A' 'A' 'T' 'A'
 'T' 'C' 'C' 'C' 'A' 'T' 'T' 'T' 'T' 'C' 'A' 'G' 'C' 'G' 'C' 'A' 'A' 'A'
 'A' 'T' 'C' 'G' 'A']
1570 ['C' 'A' 'T' 'T' 'T' 'T' 'C' 'A' 'G' 'C' 'G' 'C' 'A' 'A' 'A' 'A' 'T' 'C'
 'G' 'A' 'C' 'T' 'G' 'G' 'A' 'G' 'A' 'C' 'T' 'A' 'G' 'G' 'A' 'A' 'A' 'A'
 'T' 'C' 'G' 'T' 'C']
1571 ['A' 'A' 'A' 'T' 'A' 'A' 'A' 'T' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'T' 'T'
 'G' 'T' 'C' 'C' 'T' 'C' 'G' 'A' 'G' 'G' 'A' 'T' 'C' 'C' 'T' 'C' 'C' 'G'
 'G' 'A' 'G' 'C' 'A']
1572 ['C' 'T' 'C' 'G' 'A' 'G' 'G' 'A' 'T' 'C' 'C' 'T' 'C' 'C' 'G' 'G' 'A' 'G'
 'C' 'G' 'C' 'G' 'T' 'C' 'G' 'A' 'A' 'T' 'C' 'A' 'A' 'T' 'G' 'T' 'T' 'T'
 'C' 'C' 'G' 'G' 'C']
1573 ['A' 'T' 'G' 'A' 'T' 'T' 'G' 'T' 'A' 'G' 'A' 'A' 'C' 'C' 'C' 'T' 'T' 'T'
 'C' 'G' 'C' 'T' 'A' 'T' 'T' 'G' 'A

 'T' 'T' 'C' 'C' 'G']
2175 ['T' 'A' 'T' 'T' 'T' 'T' 'T' 'A' 'A' 'A' 'T' 'A' 'T' 'T' 'T' 'T' 'T' 'G'
 'A' 'G' 'C' 'T' 'G' 'A' 'A' 'A' 'A' 'T' 'A' 'A' 'A' 'A' 'G' 'A' 'A' 'A'
 'A' 'T' 'T' 'C' 'T']
2176 ['C' 'A' 'A' 'A' 'A' 'A' 'T' 'T' 'A' 'A' 'T' 'T' 'C' 'T' 'T' 'T' 'T' 'T'
 'T' 'T' 'C' 'G' 'C' 'T' 'A' 'A' 'A' 'A' 'A' 'T' 'T' 'G' 'A' 'G' 'A' 'A'
 'A' 'A' 'T' 'G' 'A']
2177 ['G' 'T' 'A' 'C' 'G' 'C' 'G' 'T' 'C' 'T' 'T' 'T' 'T' 'G' 'A' 'A' 'T' 'G'
 'T' 'T' 'C' 'C' 'C' 'A' 'C' 'A' 'T' 'C' 'T' 'A' 'T' 'T' 'T' 'T' 'G' 'T'
 'A' 'A' 'C' 'G' 'T']
2178 ['T' 'C' 'C' 'G' 'C' 'C' 'T' 'T' 'A' 'T' 'A' 'A' 'A' 'A' 'A' 'A' 'T' 'T'
 'G' 'C' 'C' 'A' 'C' 'T' 'T' 'C' 'C' 'G' 'G' 'C' 'A' 'A' 'A' 'T' 'T' 'G'
 'A' 'T' 'A' 'T' 'T']
2179 ['A' 'C' 'T' 'T' 'C' 'C' 'G' 'G' 'C' 'A' 'A' 'A' 'T' 'T' 'G' 'A' 'T' 'A'
 'T' 'C' 'C' 'G' 'A' 'C' 'A' 'T' 'A' 'C' 'G' 'G' 'C' 'A' 'A' 'A' 'T' 'C'
 'G' 'G' 'C' 'G' 'C']
2180 ['G' 'T' 'A' 'T' 'T' 'T' 'T' 'A' 'A' 'T' 'A' 'C' 'A' 'G' 'T' 'T' 'T' 'T'
 'G' 'A' 'C' 'G' 'T' 'C' 'A' 'T' 'T

 'A' 'A' 'C' 'T' 'A']
2707 ['T' 'A' 'T' 'T' 'C' 'T' 'G' 'G' 'T' 'A' 'T' 'C' 'C' 'A' 'T' 'A' 'G' 'C'
 'A' 'A' 'C' 'T' 'C' 'C' 'C' 'C' 'G' 'A' 'G' 'C' 'T' 'T' 'T' 'T' 'G' 'G'
 'C' 'T' 'C' 'A' 'A']
2708 ['C' 'G' 'C' 'G' 'C' 'A' 'C' 'G' 'C' 'C' 'C' 'T' 'A' 'T' 'A' 'C' 'T' 'A'
 'T' 'A' 'C' 'C' 'T' 'T' 'T' 'A' 'C' 'T' 'A' 'T' 'T' 'A' 'T' 'T' 'A' 'A'
 'A' 'T' 'G' 'C' 'T']
2709 ['C' 'T' 'T' 'T' 'A' 'C' 'T' 'A' 'T' 'T' 'A' 'T' 'T' 'A' 'A' 'A' 'T' 'G'
 'C' 'A' 'C' 'T' 'C' 'A' 'A' 'C' 'A' 'A' 'A' 'T' 'C' 'C' 'T' 'A' 'T' 'C'
 'C' 'T' 'G' 'C' 'C']
2710 ['C' 'A' 'A' 'T' 'T' 'G' 'G' 'C' 'C' 'G' 'A' 'T' 'C' 'A' 'A' 'T' 'T' 'G'
 'T' 'C' 'C' 'G' 'A' 'A' 'T' 'T' 'G' 'C' 'C' 'A' 'C' 'A' 'A' 'T' 'A' 'T'
 'G' 'C' 'A' 'C' 'T']
2711 ['G' 'T' 'G' 'C' 'C' 'A' 'C' 'G' 'T' 'T' 'G' 'G' 'G' 'G' 'G' 'A' 'A' 'A'
 'C' 'G' 'C' 'C' 'A' 'A' 'A' 'G' 'A' 'A' 'A' 'A' 'C' 'G' 'A' 'G' 'T' 'A'
 'G' 'G' 'G' 'A' 'A']
2712 ['A' 'T' 'T' 'C' 'G' 'A' 'A' 'A' 'A' 'T' 'A' 'A' 'A' 'T' 'A' 'A' 'A' 'T'
 'T' 'T' 'C' 'A' 'A' 'A' 'A' 'A' 'T

In [2]:
pd.DataFrame(final_feature_matrix).to_csv('BKF.csv',header=None,index=False)

In [1]:
final_feature_matrix

NameError: name 'final_feature_matrix' is not defined